In [1]:
import time

import googlemaps
import pandas as pd
import pymongo

from domus_analytica.config import DomusSettings

config = DomusSettings(_env_file=".env.local")
domus_db = pymongo.MongoClient(config.mongo_uri).get_database(config.mongo_db_name)
gmaps = googlemaps.Client(key=config.google_api_key)

In [2]:
# Sampled some points in fukuoka
SAMPLE_POINTS_FUKUOKA_STR = """33.58411329732284, 130.24647273566035
33.566926285036914, 130.3263123068401
33.547249980752646, 130.35440700942175
33.57962632829354, 130.36128124515983
33.5477481703356, 130.39445690459132
33.570991902675104, 130.4032803554425
33.56043973747545, 130.4524655663517
33.61279871253523, 130.41788428436948
33.63507843097913, 130.43858256993548
33.69243268074716, 130.4350487162268
33.682771273719, 130.40071985236125
33.65693217800206, 130.3371104869633"""
SAMPLE_POINTS_FUKUOKA = [
    [float(s.strip()) for s in row.split(",")]
    for row in SAMPLE_POINTS_FUKUOKA_STR.split("\n")
]
SAMPLE_POINTS_FUKUOKA

[[33.58411329732284, 130.24647273566035],
 [33.566926285036914, 130.3263123068401],
 [33.547249980752646, 130.35440700942175],
 [33.57962632829354, 130.36128124515983],
 [33.5477481703356, 130.39445690459132],
 [33.570991902675104, 130.4032803554425],
 [33.56043973747545, 130.4524655663517],
 [33.61279871253523, 130.41788428436948],
 [33.63507843097913, 130.43858256993548],
 [33.69243268074716, 130.4350487162268],
 [33.682771273719, 130.40071985236125],
 [33.65693217800206, 130.3371104869633]]

In [4]:
def places_nearby(**kwargs):
    result = gmaps.places_nearby(**kwargs)
    yield from result["results"]
    if kwargs.get("radius") is not None:
        while "next_page_token" in result:
            try:
                result = gmaps.places_nearby(page_token=result["next_page_token"])
                yield from result["results"]
            except Exception as e:
                if str(e).find('INVALID_REQUEST') != -1:
                    time.sleep(1.0)
                else:
                    raise e

In [9]:
google_cemetery_related = domus_db.get_collection("google_cemetery_related")
google_cemetery_related.create_index("place_id", unique=True)
for lat, lon in SAMPLE_POINTS_FUKUOKA:
    for doc in places_nearby(
            location=(lat, lon),
            type=["cemetery", "funeral_home"],
            radius=10000,
            language="ja",
    ):
        google_cemetery_related.update_one({"place_id": doc["place_id"]}, {"$set": doc}, upsert=True)

In [ ]:
google_cemetery_related = domus_db.get_collection("google_cemetery_related")
google_cemetery_related.create_index("place_id", unique=True)
for lat, lon in SAMPLE_POINTS_FUKUOKA:
    for doc in places_nearby(
            location=(lat, lon),
            type=["cemetery", "funeral_home"],
            radius=5000,
            language="ja",
    ):
        google_cemetery_related.update_one({"place_id": doc["place_id"]}, {"$set": doc}, upsert=True)

In [26]:
df = pd.read_csv("data/bouryokudan.csv", index_col="番号")
df

,名称,主たる事務所の所在地,代表する者
番号,,,
1,六代目山口組,兵庫県神戸市灘区篠原本町４－３－１,篠田 建市
2,稲川会,東京都港区六本木７－８－４,辛 炳圭
3,住吉会,東京都港区赤坂６－４－２１,小川 修
4,五代目工藤會,福岡県北九州市小倉北区宇佐町１－８－８,野村 悟
5,旭琉會,沖縄県中頭郡北中城村字島袋１３６２,永山 克博
6,七代目会津小鉄会,京都府京都市左京区一乗寺塚本町２１－４,金 元
7,六代目共政会,広島県広島市南区南大河町１８－１０,荒瀬 進
8,七代目合田一家,山口県下関市竹崎町３－１３－６,金 教煥
9,四代目小桜一家,鹿児島県鹿児島市甲突町９－２４,平岡 喜榮


In [30]:
japan_mafia = domus_db.get_collection("japan_mafia")
japan_mafia.create_index("place_id", unique=True)
for index, row in df.iterrows():
    address = row["主たる事務所の所在地"]
    for doc in gmaps.geocode(address=address, language="ja", region="ja"):
        doc["mafia_boss"] = row["代表する者"]
        doc["mafia_name"] = row["名称"]
        japan_mafia.update_one({"place_id": doc["place_id"]}, {"$set": doc}, upsert=True)